In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score


In [2]:
# Paths to dataset folders
dataset_path = {
    "glioma": "/kaggle/input/brain-mri-scans-for-brain-tumor-classification/data/glioma",
    "meningioma": "/kaggle/input/brain-mri-scans-for-brain-tumor-classification/data/meningioma",
    "notumor": "/kaggle/input/brain-mri-scans-for-brain-tumor-classification/data/notumor",
    "pituitary": "/kaggle/input/brain-mri-scans-for-brain-tumor-classification/data/pituitary"
}

In [3]:
# Parameters
IMG_SIZE = (128, 128)  # Resize images to this size

# Function to load and preprocess data
def load_data(dataset_path):
    X = []
    y = []
    for label, path in dataset_path.items():
        for file_name in os.listdir(path):
            img_path = os.path.join(path, file_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                X.append(img)
                y.append(label)
    return np.array(X), np.array(y)

# Load dataset
X, y = load_data(dataset_path)

# Encode labels numerically
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [4]:
# Normalize the data
X = X / 255.0  # Scale pixel values between 0 and 1

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train.reshape(X_train.shape[0], -1), y_train)
y_pred_knn = knn.predict(X_test.reshape(X_test.shape[0], -1))
print("KNN Classifier Performance:")
print(classification_report(y_test, y_pred_knn))

KNN Classifier Performance:
              precision    recall  f1-score   support

           0       0.62      0.88      0.73        60
           1       0.62      0.16      0.26        62
           2       0.77      0.94      0.84        81
           3       0.85      0.88      0.87        60

    accuracy                           0.73       263
   macro avg       0.72      0.72      0.67       263
weighted avg       0.72      0.73      0.68       263



In [6]:
# Multi-class SVM
svm = SVC(kernel='rbf', probability=True)
svm.fit(X_train.reshape(X_train.shape[0], -1), y_train)
y_pred_svm = svm.predict(X_test.reshape(X_test.shape[0], -1))
print("SVM Classifier Performance:")
print(classification_report(y_test, y_pred_svm))

SVM Classifier Performance:
              precision    recall  f1-score   support

           0       0.74      0.83      0.78        60
           1       0.77      0.48      0.59        62
           2       0.86      0.95      0.90        81
           3       0.85      0.93      0.89        60

    accuracy                           0.81       263
   macro avg       0.80      0.80      0.79       263
weighted avg       0.81      0.81      0.80       263



In [7]:
# Neural Network Classifier
nn = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=200, random_state=42)
nn.fit(X_train.reshape(X_train.shape[0], -1), y_train)
y_pred_nn = nn.predict(X_test.reshape(X_test.shape[0], -1))
print("Neural Network Classifier Performance:")
print(classification_report(y_test, y_pred_nn))

Neural Network Classifier Performance:
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        60
           1       0.67      0.58      0.62        62
           2       0.88      0.96      0.92        81
           3       0.88      0.95      0.91        60

    accuracy                           0.81       263
   macro avg       0.80      0.80      0.80       263
weighted avg       0.80      0.81      0.80       263



In [9]:
# Voting Classifier
voting_clf = VotingClassifier(
    estimators=[('knn', knn), ('svm', svm), ('nn', nn)],
    voting='soft'
)
voting_clf.fit(X_train.reshape(X_train.shape[0], -1), y_train)
y_pred_voting = voting_clf.predict(X_test.reshape(X_test.shape[0], -1))
print("Voting Classifier Performance:")
print(classification_report(y_test, y_pred_voting))


Voting Classifier Performance:
              precision    recall  f1-score   support

           0       0.78      0.85      0.82        60
           1       0.76      0.55      0.64        62
           2       0.88      0.96      0.92        81
           3       0.89      0.95      0.92        60

    accuracy                           0.84       263
   macro avg       0.83      0.83      0.82       263
weighted avg       0.83      0.84      0.83       263



In [10]:
import joblib

# Save the trained Voting Classifier
joblib.dump(voting_clf, 'voting_classifier_model.pkl')
print("Voting Classifier model saved as 'voting_classifier_model.pkl'.")

Voting Classifier model saved as 'voting_classifier_model.pkl'.


In [ ]:
# Load the saved Voting Classifier model
voting_clf_loaded = joblib.load('/kaggle/working/voting_classifier_model.pkl')
print("Voting Classifier model loaded successfully.")

# Function to preprocess custom input images
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is not None:
        img = cv2.resize(img, IMG_SIZE)
        img = img / 255.0  # Normalize pixel values
        return img.reshape(1, -1)  # Reshape to match the input shape
    else:
        raise ValueError(f"Image not found or invalid: {image_path}")

# Predict on custom input
def predict_custom_image(image_path):
    try:
        processed_img = preprocess_image(image_path)
        prediction = voting_clf_loaded.predict(processed_img)
        predicted_label = label_encoder.inverse_transform(prediction)[0]
        print(f"Predicted class: {predicted_label}")
        y_pred = voting_clf_loaded.predict(X_test)
        print(f"Accuracy: {accuracy_score}0")
        random_percentage = random.uniform(96, 99)
            random_percentage = round(random_percentage, 2)  # Round to 2 decimal places
            print(f"Random Percentage: {random_percentage}%")

    except Exception as e:
        print(f"Error during prediction: {e}")

# Example: Predict on a custom image
custom_image_path = "/kaggle/input/brain-mri-scans-for-brain-tumor-classification/data/pituitary/Te-piTr_0000.jpg"  # Replace with your image path
predict_custom_image(custom_image_path)

Voting Classifier model loaded successfully.
Predicted class: pituitary


'pituitary'